# Sentiment Analysis:

Naive Bayez works great on textual data.

### Work Flow:

1. Text Preprocessing: Remove Html tags, common words, remove special characeters.
2. Vectorize(BOW) the data.
3. Pass data to algorithm and train.
4. Deployement.



## Bag Of Words:

Find all unique words. For example we have following 4 words:-

Each new row represent a review and it value contains occurennce of that word in a review.

<table>

<tr>
<td>Great</td>
<td>awesome</td>
<td>poor</td>
<td>pathetic</td>
</tr>

<tr>
<td>3</td>
<td>1</td>
<td>0</td>
<td>0</td>
</tr>

<tr>
<td>0</td>
<td>1</td>
<td>2</td>
<td>4</td>
</tr>


</table>


Shape will become (50000, 4)

50000 -> Number of reviews

4 -> Number of unique words

## Apply bayes theorem now.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import nltk # For StopWords
from nltk.corpus import stopwords # For Stopwords
from nltk.stem.porter import PorterStemmer # For stemming words
from sklearn.feature_extraction.text import CountVectorizer # For Vectorizing words
from sklearn.model_selection import train_test_split # For test train split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Model Implementation

In [22]:
data = pd.read_csv("Dataset/IMDB Dataset.csv")
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [23]:
data = data.sample(50000).reset_index()
data.shape

(50000, 3)

In [24]:
# One Review
data['review'][0]

'IF ANYONE IS INTERESTED IN OBTAINING A COPY OF THIS FILM PLEASE READ THE BOTTOM OF THIS DESCRIPTION: First telecast by CBS on November 30, 2003, the made-for-TV Finding John Christmas is a sequel to the previous year\'s A Town Without Christmas, with Peter Falk reprising his role as versatile guardian angel Max. Valerie Bertinelli plays Kathleen McAllister, a divorced small-town nurse whose depression... over the fact that the hospital ER she maintains may be forced to shut down because of a $100,000 debt is briefly lifted when she spots a newspaper picture taken by photojournalist Noah Greeley (David Cubitt). The picture shows an act of bravery performed by Noah\'s firefighter brother Hank (William Russ), who mysteriously left town 25 years ago and hasn\'t been seen since. Hank would like to quietly slip back into town without explanation or fanfare, but this proves impossible when Noah\'s newspaper posts a $50,000 reward to identify Hank, known only to the public as "John Christmas.

## Text Cleaning:

1. Remove HTML Tags.
2. Remove special characters.
3. Convert everything to lower case.
4. Removing Stop Words.
5. Stemming (One words used in multiple form, example play, playing, played, etc)

In [25]:
# No missing value in data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      50000 non-null  int64 
 1   review     50000 non-null  object
 2   sentiment  50000 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [26]:
# Changing Postive Negative to 0 and 1.
data['sentiment'].replace({'positive': 1, 'negative': 0}, inplace=True)

In [27]:
data.head()

,index,review,sentiment
0,25671,IF ANYONE IS INTERESTED IN OBTAINING A COPY OF...,1
1,14406,Tug-3 is absolutely right. Although I am sure ...,0
2,31152,I just saw this early this morning on the Fox ...,0
3,35568,I just don't get it. Why call this a sequel to...,0
4,32247,This film easily rivals the emotional strength...,1


### Removing HTML Tags

In [28]:
data.iloc[2].review

'I just saw this early this morning on the Fox channel quite by accident (my dog woke me up) - I had seen it years ago and thought I remembered it fairly well. As a kid, I had enjoyed it. But now? As another poster commented, several of the reels were out of order, and while it was disorienting at first, and bizarre, it seemed to fit the production - what was just awful became surreal and amusing. Musical numbers for what I think was the "big" fundraiser show("you\'re in show business, I\'m in show business, most of the kids are in show business, let\'s put on a show")come out of nowhere BEFORE all the talk about putting on a show, and then fade without applause to totally unrelated "straight" scenes. The leading man\'s girlfriend shows up, spits out lines and lines of dialogue, then disappears. I was half awake, and loved every insane minute of it.'

In [29]:
# Removing all HTML Tags
clean = re.compile('<.*?>')
re.sub(clean, '', data.iloc[2]. review)

'I just saw this early this morning on the Fox channel quite by accident (my dog woke me up) - I had seen it years ago and thought I remembered it fairly well. As a kid, I had enjoyed it. But now? As another poster commented, several of the reels were out of order, and while it was disorienting at first, and bizarre, it seemed to fit the production - what was just awful became surreal and amusing. Musical numbers for what I think was the "big" fundraiser show("you\'re in show business, I\'m in show business, most of the kids are in show business, let\'s put on a show")come out of nowhere BEFORE all the talk about putting on a show, and then fade without applause to totally unrelated "straight" scenes. The leading man\'s girlfriend shows up, spits out lines and lines of dialogue, then disappears. I was half awake, and loved every insane minute of it.'

In [30]:
# Function to clean HTML Tags:
def clean_html(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [31]:
data['review'] = data['review'].apply(clean_html)

In [32]:
data.iloc[2].review

'I just saw this early this morning on the Fox channel quite by accident (my dog woke me up) - I had seen it years ago and thought I remembered it fairly well. As a kid, I had enjoyed it. But now? As another poster commented, several of the reels were out of order, and while it was disorienting at first, and bizarre, it seemed to fit the production - what was just awful became surreal and amusing. Musical numbers for what I think was the "big" fundraiser show("you\'re in show business, I\'m in show business, most of the kids are in show business, let\'s put on a show")come out of nowhere BEFORE all the talk about putting on a show, and then fade without applause to totally unrelated "straight" scenes. The leading man\'s girlfriend shows up, spits out lines and lines of dialogue, then disappears. I was half awake, and loved every insane minute of it.'

### Converting everything to lower

In [33]:
def convert_lower(text):
    return text.lower()

In [34]:
data['review'] = data['review'].apply(convert_lower)

In [35]:
data.head()

,index,review,sentiment
0,25671,if anyone is interested in obtaining a copy of...,1
1,14406,tug-3 is absolutely right. although i am sure ...,0
2,31152,i just saw this early this morning on the fox ...,0
3,35568,i just don't get it. why call this a sequel to...,0
4,32247,this film easily rivals the emotional strength...,1


### Remove Special Characters

In [36]:
def remove_special(text):
    x = ""
    
    for i in text:
        if i.isalnum():
            x = x + i
        else:
            x = x + " "
    return x

In [37]:
data.iloc[2].review

'i just saw this early this morning on the fox channel quite by accident (my dog woke me up) - i had seen it years ago and thought i remembered it fairly well. as a kid, i had enjoyed it. but now? as another poster commented, several of the reels were out of order, and while it was disorienting at first, and bizarre, it seemed to fit the production - what was just awful became surreal and amusing. musical numbers for what i think was the "big" fundraiser show("you\'re in show business, i\'m in show business, most of the kids are in show business, let\'s put on a show")come out of nowhere before all the talk about putting on a show, and then fade without applause to totally unrelated "straight" scenes. the leading man\'s girlfriend shows up, spits out lines and lines of dialogue, then disappears. i was half awake, and loved every insane minute of it.'

In [38]:
remove_special(data.iloc[2].review)

'i just saw this early this morning on the fox channel quite by accident  my dog woke me up    i had seen it years ago and thought i remembered it fairly well  as a kid  i had enjoyed it  but now  as another poster commented  several of the reels were out of order  and while it was disorienting at first  and bizarre  it seemed to fit the production   what was just awful became surreal and amusing  musical numbers for what i think was the  big  fundraiser show  you re in show business  i m in show business  most of the kids are in show business  let s put on a show  come out of nowhere before all the talk about putting on a show  and then fade without applause to totally unrelated  straight  scenes  the leading man s girlfriend shows up  spits out lines and lines of dialogue  then disappears  i was half awake  and loved every insane minute of it '

In [39]:
data['review'] = data['review'].apply(remove_special)
data.head()

,index,review,sentiment
0,25671,if anyone is interested in obtaining a copy of...,1
1,14406,tug 3 is absolutely right although i am sure ...,0
2,31152,i just saw this early this morning on the fox ...,0
3,35568,i just don t get it why call this a sequel to...,0
4,32247,this film easily rivals the emotional strength...,1


### Remove Stop Words

In [40]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [41]:
def remove_stopwords(text): 
    x = []
    for i in text.split():
        if i not in stopwords.words('english'):
            x.append(i)
    y = x[:]
    x.clear()
    return y

In [42]:
data.iloc[2].review

'i just saw this early this morning on the fox channel quite by accident  my dog woke me up    i had seen it years ago and thought i remembered it fairly well  as a kid  i had enjoyed it  but now  as another poster commented  several of the reels were out of order  and while it was disorienting at first  and bizarre  it seemed to fit the production   what was just awful became surreal and amusing  musical numbers for what i think was the  big  fundraiser show  you re in show business  i m in show business  most of the kids are in show business  let s put on a show  come out of nowhere before all the talk about putting on a show  and then fade without applause to totally unrelated  straight  scenes  the leading man s girlfriend shows up  spits out lines and lines of dialogue  then disappears  i was half awake  and loved every insane minute of it '

In [43]:
remove_stopwords(data.iloc[2].review)

['saw',
 'early',
 'morning',
 'fox',
 'channel',
 'quite',
 'accident',
 'dog',
 'woke',
 'seen',
 'years',
 'ago',
 'thought',
 'remembered',
 'fairly',
 'well',
 'kid',
 'enjoyed',
 'another',
 'poster',
 'commented',
 'several',
 'reels',
 'order',
 'disorienting',
 'first',
 'bizarre',
 'seemed',
 'fit',
 'production',
 'awful',
 'became',
 'surreal',
 'amusing',
 'musical',
 'numbers',
 'think',
 'big',
 'fundraiser',
 'show',
 'show',
 'business',
 'show',
 'business',
 'kids',
 'show',
 'business',
 'let',
 'put',
 'show',
 'come',
 'nowhere',
 'talk',
 'putting',
 'show',
 'fade',
 'without',
 'applause',
 'totally',
 'unrelated',
 'straight',
 'scenes',
 'leading',
 'man',
 'girlfriend',
 'shows',
 'spits',
 'lines',
 'lines',
 'dialogue',
 'disappears',
 'half',
 'awake',
 'loved',
 'every',
 'insane',
 'minute']

In [ ]:
data['review'] = data['review'].apply(remove_stopwords)

### Perform Stemming

In [ ]:
ps = PorterStemmer()

In [ ]:
y = []
def stem_words(text):
    for i in text:
        y.append(ps.stem(i))
    z = y[:]
    y.clear()
    return z

In [ ]:
data['review'] = data['review'].apply(stem_words)
data.head()

In [ ]:
# Join Back
def join_back(list_input):
    return " ".join(list_input)

In [ ]:
data['review'] = data['review'].apply(join_back)
data.head()

## Final Preparations:

To get all words used in review

In [46]:
max_f = -1

In [ ]:
for i in range(100, 10000, 100):
    
    cv= CountVectorizer(max_features=i)
    
    x = cv.fit_transform(data['review']).toarray()
    y = data.iloc[:,-1].values

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)
        
    clf1 = GaussianNB()
    clf2 = MultinomialNB()
    clf3 = BernoulliNB()

    clf1.fit(x_train, y_train)
    clf2.fit(x_train, y_train)
    clf3.fit(x_train, y_train)

    y_pred1 = clf1.predict(x_test)
    y_pred2 = clf2.predict(x_test)
    y_pred3 = clf3.predict(x_test)

    a_score1 = accuracy_score(y_test, y_pred1)
    a_score2 = accuracy_score(y_test, y_pred2)
    a_score3 = accuracy_score(y_test, y_pred3)

    if(a_score1 > max_f):
        print(i)
        print("Gaussian:", a_score1)
        max_f = a_score1
        print()

    if(a_score2 > max_f):
        print(i)
        print("Multinomial:", a_score2)
        max_f = a_score2
        print()

    if(a_score3 > max_f):
        print(i)
        print("Bernaulli:", a_score3)
        max_f = a_score3
        print()

100
Gaussian: 0.7097

100
Multinomial: 0.7342

200
Gaussian: 0.7577

200
Multinomial: 0.7729

300
Gaussian: 0.7862

300
Multinomial: 0.8048

400
Multinomial: 0.8142

500
Multinomial: 0.83

800
Bernaulli: 0.8304

900
Bernaulli: 0.8325

1000
Multinomial: 0.8327

1000
Bernaulli: 0.8397

1200
Bernaulli: 0.8432

1300
Bernaulli: 0.8449

1800
Bernaulli: 0.8472

2000
Bernaulli: 0.8501

2600
Bernaulli: 0.8527

4000
Bernaulli: 0.8537



In [75]:
cv= CountVectorizer()

In [76]:
x = cv.fit_transform(data['review']).toarray()

In [77]:
x.shape

(10000, 36394)

In [78]:
y = data.iloc[:,-1].values

In [79]:
y.shape

(10000,)

In [80]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [81]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(8000, 36394)
(8000,)
(2000, 36394)
(2000,)


### Preparing model

In [82]:
clf1 = GaussianNB()
clf2 = MultinomialNB()
clf3 = BernoulliNB()

In [83]:
clf1.fit(x_train, y_train)
clf2.fit(x_train, y_train)
clf3.fit(x_train, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [84]:
y_pred1 = clf1.predict(x_test)
y_pred2 = clf2.predict(x_test)
y_pred3 = clf3.predict(x_test)

In [86]:
y_pred1.shape

(2000,)

In [92]:
print("Gaussian:", accuracy_score(y_test, y_pred1))
print("Multinomial:", accuracy_score(y_test, y_pred2))
print("Bernaulli:", accuracy_score(y_test, y_pred3))

Gaussian: 0.6155


In [2]:
import pickle

In [5]:
loaded_model = pickle.load(open("models/model", 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)

In [6]:
load_text = pickle.load(open("models/words", "rb"))

In [20]:
x = load_text.transform(["worst movie ever"])

In [21]:
loaded_model.predict(x)

array([0])